<a id="stepSTART"></a>

**1. Вопросы исследования**: 
* Есть ли основания полагать, что покупка пользователя обеспечивается активностями за рамками приложения, и вкладк самого приложения минимален
* Определить, сколько времени проходит до первой покупки
* Определить, сколько времени проходит до первой покупки тарифа
* Определить, какие покупки предшествуют покупке тарифа

**2. Ключевые выводы**: 

2.1 Ключевой фактор первой покупки:

Ключевым фактором влияющим на первую покупку является "активность" за рамками приложения, ввиду того, что преимущественно проходит ноль дней от регистрации в приложении до первой покупки. 

2.2 Кол-во дней до первой покупки:

2.2.2 Наиболее часто встречающееся значение: 0 дней, медианное — 5-7 дней, в зависимости от статуса пользователя.
2.2.3 Наиболее часто встречающееся значение у Гостей  — 2, вероятнее всего связано с тем, что Гость, это будущий Эксперт, и 2 дня занимают процедуры по оформлению статуса.

2.3 Кол-во дней до первой покупки тарифа:

Следует исходить из того, что преимущественно тариф приобретается сразу после регистрации в приложении, а если исходить из контекста, то его покупка происходит до регистрации в приложении.

2.3 Паттерны покупок
* Путь Амбассадора и Эксперта похожи, это связано с переходом от Амбассадора к Эксперту.
* Амбассадоры и Эксперты – самые предсказуемые: они заходят через свои ключевые тарифы.
* Амбассадоры второй покупкой также приобретают тариф, но  здесь уже видно, что дальнейшие покупки связаны с расширением участия в экосистеме.
* Эксперты приобретают экспертный тариф и во второй покупке нет оснований говорить о желании расширения участия в экосистеме.
* Эксперты редко переходят к Амбассадорам.
* Гости — первый выбор хаотичен, но с незначительным отрывом идет выбор "Эксперт".
* Жители, как правило, остаются на подписке и не имеют интереса к расширению в экосистеме.
* Жители – делают ставку на долгосрочные подписки, а не разовые покупки.

2.4 "Апгрейд" тарифа:
* Данные ограничены, но есть основания, что нельзя сказать, о наличии сценария, при котором Гость, Житель, Эксперт переходят в статус Амбассадора. В текущей комбинации бизнес-модели, "апгрейда" статуса вероятнее всего нет. Не приходится сейчас рассчитывать на то, что Гость, Житель, Эксперт в дальнейшем перейдут к статусу Амбассадор.

2.4 Влияние мероприятия

При первом "приближении" можно сделать вывод, что мероприятие является сильным стимулирующим фактором к покупке тарифа/

**3. Рекомендации**: 
* Продолжительность коммуникации с целью онбординга пользователя лучше делать максимум 7 дней.
* Расширять количество мероприятий и работать над факторами, стимулирующими покупку тарифа на самом мероприятии.
* Переработать предложение, которое станет привлекательным для "апгрейда" тарифа у Гостей, Жителей и Экспертов — и SPACE Pass может стать "мостом" к этому, при "вознаграждении" продажи у Амбассадоров.


**4. Источник данные**: 
* Использованы данные по покупкам в системе с октября 2024 по март 2025. Допущение — нет возможности отследить историю изменения статуса.

**5. Инструменты**: 
* В анализе использовал библиотеки Python для проведения исследовательского анализа данных. Кроме того, использовал SQL для получения данных приложения

In [1097]:
# Импорт необходимых библиотек и формирование датафрейма для исследования
import pandas as pd
import mysql.connector
import re
from sqlalchemy import create_engine

# 🔹 Полный путь к файлу (замените на свой)
file_path = r"C:\Users\come_on_dude\come_on_dude\come_on_dude\come_on_dude\come_on_dude.csv"  

# 🔹 Формирование датафрейма
ambassadors = pd.read_csv(file_path)

In [1098]:
# Сводная информация по датафрейму
ambassadors.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5961 entries, 0 to 5960
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   name         5960 non-null   object
 1   phone        5960 non-null   object
 2   tariff       5961 non-null   object
 3   sale_status  5961 non-null   object
 4   sale_date    5961 non-null   object
dtypes: object(5)
memory usage: 233.0+ KB


In [ ]:
# Статистика набора данных
ambassadors.describe()

In [1099]:
# Посмотрим нулевые значения в столбцах содержащих их
ambassadors[ambassadors.isnull().any(axis=1)]

,name,phone,tariff,sale_status,sale_date
4838,NaN,NaN,Professional - рассрочка,Исполнено,29.10.2024 6:44


In [1100]:
# Удалим нулевые значения в столбцах содержащих их
ambassadors = ambassadors.dropna()

In [1101]:
# Проверим набор данных на наличие дубликатов
duplicates = ambassadors.duplicated()
print(f"Набор данных содержит {duplicates.sum()} одинаковых значения")

Набор данных содержит 947 одинаковых значения


In [1102]:
# Удалим дублирующиеся строки по определенным столбцам, оставив первую строку
ambassadors = ambassadors.sort_values(by=["phone", "sale_date"], ascending=[True, False])
ambassadors = ambassadors.drop_duplicates(subset=["name","phone", "tariff", "sale_date"], keep="first")

In [1103]:
# Уберем тестовые строки
ambassadors = ambassadors[~ambassadors["name"].str.contains(r"Тест|тест|test", case=False, na=False, regex=True)]

In [1104]:
# Уберем пробелы в номере телефона
ambassadors['phone'] = ambassadors['phone'].str.replace(" ", "")

In [1105]:
# Уберем тестовые строки, определив их по кол-ву символов в номере телефона
pattern = r"^\d{7,}$"
ambassadors = ambassadors[ambassadors['phone'].apply(lambda x: bool(re.match(pattern, x)))]

In [1106]:
# Приведем названия тарифов к одному значению
replace_values = [
    "Standard - рассрочка", "Standard - b2b", "Professional - b2b", "Professional - рассрочка",
    "Standard - кредит", "Professional - кредит", "Professional", "Standard",
    "Standard - иностранный пользователь", "Professional - иностранный пользователь",
    "Амбассадор - иностранный пользователь"
]

# Сделаем замену названия тарифов 
ambassadors.loc[:, "tariff"] = ambassadors["tariff"].apply(
    lambda x: "Покупка тарифа Амбассадор" if x in replace_values else x
)

In [1107]:
# Сделаем запрос к базе данных приложения, чтобы иметь данные по дате регистрации в приложении
engine = create_engine('mysql+mysqlconnector://come_on_dude')

query = """SELECT phone,
                  CASE 
                      WHEN clubMemberStatus = "MEMBER" THEN 'Герой'
                      WHEN clubMemberStatus = "LEAD" THEN 'Житель'
                      WHEN clubMemberStatus IN ("EXPERT", "EXPERT_PRO") THEN 'Эксперт'
                      WHEN clubMemberStatus = "AMBASSADOR" THEN 'Амбассадор'
                      WHEN clubMemberStatus = "GUEST" THEN 'Гость'
                      ELSE 'не определено'
                  END AS status,
                  createdAt AS reg_date
            FROM `come_on_dude`.users AS users
           WHERE id NOT IN (SELECT id AS authorId
                              FROM `come_on_dude`.users
                             WHERE (phone IN ('come_on_dude',
                                              'come_on_dude',
                                              'come_on_dude',
                                              'come_on_dude',
                                              'come_on_dude',
                                              'come_on_dude',
                                              'come_on_dude',
                                              'come_on_dude',
                                              'come_on_dude',
                                              'come_on_dude',
                                              'come_on_dude')
                                    OR externalId = 'come_on_dude')
                            )
            AND DATE(createdAt) >= "2024-10-01" """

# Сформируем датафрейм
users = pd.read_sql(query, engine)

In [1108]:
# Посмотрим, есть ли в датафрейме users номера с символами и пробелами
wrong_phone = users[users["phone"].str.contains(r"[+ ]", regex=True)]
wrong_phone

,phone,status,reg_date


In [1109]:
# Соеденим датафреймы и удалим тех, у кого отсутствует дата регистрации
ambassadors = ambassadors.merge(users, on="phone", how="left")
ambassadors = ambassadors.dropna(subset=["reg_date"])

In [1110]:
# Преобразуем даты
ambassadors["reg_date"] = pd.to_datetime(ambassadors["reg_date"], format="%d.%m.%Y %H:%M", errors="coerce")
ambassadors["sale_date"] = pd.to_datetime(ambassadors["sale_date"], format="%d.%m.%Y %H:%M", errors="coerce")
ambassadors["reg_date"] = pd.to_datetime(ambassadors["reg_date"], dayfirst=True).dt.floor('s')
ambassadors["sale_date"] = pd.to_datetime(ambassadors["sale_date"], dayfirst=True).dt.floor('s')

In [1111]:
# Удалим дубликаты по одной сделке, оставив только первый статус
ambassadors = ambassadors.sort_values(by="sale_status", ascending=False)
ambassadors = ambassadors.drop_duplicates(subset=["phone", "tariff", "sale_date"], keep="first")

In [1112]:
# Уберем колонку 
ambassadors = ambassadors.drop(columns=["sale_status"])

In [1113]:
# Уберем пользователей, которые регистрировались ранее, чем есть данные по покупкам

# Для каждого пользователя оставим только первую покупку по дате
first_purchase = ambassadors.groupby("phone")["sale_date"].min()

# Присоединим к датафрейму информацию о первой покупке
ambassadors = ambassadors.merge(first_purchase, on="phone", suffixes=("", "_first_purchase"))

# Оставляем только пользователей, у которых reg_date меньше или равна первой покупке
ambassadors = ambassadors[ambassadors["reg_date"] <= ambassadors["sale_date_first_purchase"]]

In [1114]:
# Найдем кол-во пользователей совершивших покупку
unique_users_count = ambassadors["phone"].nunique()
print(f"Пользователей совершивших покупку {duplicates.sum()}")

Пользователей совершивших покупку 947


In [1115]:
# Найдем кол-во дней до первой покупки
ambassadors_first_purchase = ambassadors.copy()
ambassadors_first_purchase["days_to_first_purchase"] = \
    (ambassadors_first_purchase["sale_date_first_purchase"] 
    - ambassadors_first_purchase["reg_date"]).dt.days 



average_days = ambassadors_first_purchase["days_to_first_purchase"].mean()
median_days = ambassadors_first_purchase["days_to_first_purchase"].median()
max_days = ambassadors_first_purchase["days_to_first_purchase"].max()
mode_days = ambassadors_first_purchase["days_to_first_purchase"].mode()[0] 

print(f"Среднее количество дней: {round(average_days, 2)}")
print(f"Медианное количество дней: {median_days}")
print(f"Максимальное количество дней: {max_days}")
print(f"Наиболее часто встречающееся значение: {mode_days}")

Среднее количество дней: 11.34
Медианное количество дней: 5.0
Максимальное количество дней: 159
Наиболее часто встречающееся значение: 0


In [1116]:
# Найдем кол-во дней до первой покупки по статусам

# Амбассадор
ambassadors_first_purchase = ambassadors.copy()
ambassadors_first_purchase = ambassadors_first_purchase.query('status == "Амбассадор"')
ambassadors_first_purchase["days_to_first_purchase"] = \
    (ambassadors_first_purchase["sale_date_first_purchase"] \
    - ambassadors_first_purchase["reg_date"]).dt.days

average_days_ambassador = ambassadors_first_purchase["days_to_first_purchase"].mean()
median_days_ambassador = ambassadors_first_purchase["days_to_first_purchase"].median()
max_days_ambassador = ambassadors_first_purchase["days_to_first_purchase"].max()
mode_days_ambassador = ambassadors_first_purchase["days_to_first_purchase"].mode()[0] 

# Эксперт
expert_first_purchase = ambassadors.copy()
expert_first_purchase = expert_first_purchase.query('status == "Эксперт"')
expert_first_purchase["days_to_first_purchase"] = \
    (expert_first_purchase["sale_date_first_purchase"] \
    - expert_first_purchase["reg_date"]).dt.days

average_days_expert = expert_first_purchase["days_to_first_purchase"].mean()
median_days_expert = expert_first_purchase["days_to_first_purchase"].median()
max_days_expert = expert_first_purchase["days_to_first_purchase"].max()
mode_days_expert = expert_first_purchase["days_to_first_purchase"].mode()[0]  

# Гость
quest_first_purchase = ambassadors.copy()
quest_first_purchase = quest_first_purchase.query('status == "Гость"')
quest_first_purchase["days_to_first_purchase"] = \
    (quest_first_purchase["sale_date_first_purchase"] \
    - quest_first_purchase["reg_date"]).dt.days

average_days_quest = quest_first_purchase["days_to_first_purchase"].mean()
median_days_quest = quest_first_purchase["days_to_first_purchase"].median()
max_days_quest = quest_first_purchase["days_to_first_purchase"].max()
mode_days_quest = quest_first_purchase["days_to_first_purchase"].mode()[0]  

# Житель
citizen_first_purchase = ambassadors.copy()
citizen_first_purchase = citizen_first_purchase.query('status == "Житель"')
citizen_first_purchase["days_to_first_purchase"] = \
    (citizen_first_purchase["sale_date_first_purchase"] \
    - citizen_first_purchase["reg_date"]).dt.days \

average_days_citizen = citizen_first_purchase["days_to_first_purchase"].mean()
median_days_citizen = citizen_first_purchase["days_to_first_purchase"].median()
max_days_citizen = citizen_first_purchase["days_to_first_purchase"].max()
mode_days_citizen = citizen_first_purchase["days_to_first_purchase"].mode()[0]  

print(f"Среднее количество дней до первой покупки: Амбассадор")
print(f"Среднее количество дней: {round(average_days_ambassador, 2)}")
print(f"Медианное количество дней: {median_days_ambassador}")
print(f"Максимальное количество дней: {max_days_ambassador}")
print(f"Наиболее часто встречающееся значение: {mode_days_ambassador}")
print()
print(f"Среднее количество дней до первой покупки: Эксперт")
print(f"Среднее количество дней: {round(average_days_expert, 2)}")
print(f"Медианное количество дней: {median_days_expert}")
print(f"Максимальное количество дней: {max_days_expert}")
print(f"Наиболее часто встречающееся значение: {mode_days_expert}")
print()
print(f"Среднее количество дней до первой покупки: Гость")
print(f"Среднее количество дней: {round(average_days_quest, 2)}")
print(f"Медианное количество дней: {median_days_quest}")
print(f"Максимальное количество дней: {max_days_quest}")
print(f"Наиболее часто встречающееся значение: {mode_days_quest}")
print()
print(f"Среднее количество дней до первой покупки: Житель")
print(f"Среднее количество дней: {round(average_days_citizen, 2)}")
print(f"Медианное количество дней: {median_days_citizen}")
print(f"Максимальное количество дней: {max_days_citizen}")
print(f"Наиболее часто встречающееся значение: {mode_days_citizen}")



Среднее количество дней до первой покупки: Амбассадор
Среднее количество дней: 11.98
Медианное количество дней: 7.0
Максимальное количество дней: 107
Наиболее часто встречающееся значение: 0

Среднее количество дней до первой покупки: Эксперт
Среднее количество дней: 10.94
Медианное количество дней: 5.0
Максимальное количество дней: 159
Наиболее часто встречающееся значение: 0

Среднее количество дней до первой покупки: Гость
Среднее количество дней: 13.53
Медианное количество дней: 6.0
Максимальное количество дней: 134
Наиболее часто встречающееся значение: 2

Среднее количество дней до первой покупки: Житель
Среднее количество дней: 16.35
Медианное количество дней: 7.0
Максимальное количество дней: 73
Наиболее часто встречающееся значение: 0


In [1117]:
# Ранижруем и пронумеруем покупки по каждому пользователю
ambassadors = ambassadors.sort_values(by=["phone", "sale_date"])
ambassadors["purchase_number"] = ambassadors.groupby("phone").cumcount() + 1

# Посмотрим распределение пользователей по кол-ву покупок
user_purchase_counts = ambassadors.groupby("phone")["purchase_number"].max()
purchase_сounts = user_purchase_counts.value_counts().sort_index()
purchase_сounts

purchase_number
1    2545
2     151
3      26
4       8
5       6
6       2
7       1
Name: count, dtype: int64

In [1118]:
# Найдем кол-во пользователей, которые купили тариф
users_buy_tariff = ambassadors.copy()
users_buy_tariff = users_buy_tariff.query('tariff == "Покупка тарифа Амбассадор"')
users_buy_tariff = users_buy_tariff["phone"].nunique()
print(f"Пользователей купивших тариф: {users_buy_tariff}")

Пользователей купивших тариф: 254


In [1119]:
# Найдем кол-во пользователей, которые купили и тариф и купили билет на мероприятие
# Отфильтруем пользователей, которые сделали покупку "Покупка тарифа Амбассадор"
ambassadors_tarif = ambassadors[ambassadors["tariff"] == "Покупка тарифа Амбассадор"]
ambassadors_tarif = ambassadors_tarif.query('sale_date <= "2025-03-04 15:08:00"')

# Отфильтруем пользователей, которые сделали покупку "Билет на Федеральное мероприятие для входа Professional"
ticket_tariffs = [
    "Билет на Федеральное мероприятие для входа Professional",
    "Билет STANDARD - Space Business Action 2025 С 20 февраля",
    "Билет STANDARD - Space Business Action 2025 С 20 января",
    "Билет VIP - Space Business Action 2025 С 20 февраля",
    "Билет STANDARD - Space Business Action 2025",
    "Билет VIP - Space Business Action 2025 С 20 января"
]
ambassadors_ticket = ambassadors[ambassadors["tariff"].isin(ticket_tariffs)]

# Объединяем оба датафрейма по phone, чтобы оставить только тех пользователей, которые есть в обоих
users_with_both = pd.merge(ambassadors_tarif, ambassadors_ticket, on="phone", how="inner", suffixes=("_ambassador", "_ticket"))

users_with_both = users_with_both["phone"].nunique()
print(f"Пользователей купивших и тариф и билет: {users_with_both}")

Пользователей купивших и тариф и билет: 42


In [1120]:
# Рассчитаем процент пользователей, купивших билет и купивших тариф
print(f"Процент пользователей купивших и тариф и билет: {round((users_with_both * 100) / users_buy_tariff, 2)}")

Процент пользователей купивших и тариф и билет: 16.54


In [1121]:
# Найдем распределение покупок в зависимости от статуса пользователя
# Амбассадор
# Отбираем только первую покупку
first_purchases_ambassador = ambassadors.copy()
first_purchases_ambassador = first_purchases_ambassador.query('status == "Амбассадор"')
first_purchases_ambassador = first_purchases_ambassador[first_purchases_ambassador["purchase_number"] == 1]
# Считаем, сколько раз встречается каждый тариф
first_distribution_ambassador = first_purchases["tariff"].value_counts()

# Эксперт
# Отбираем только первую покупку
first_purchases_expert = ambassadors.copy()
first_purchases_expert = first_purchases_expert.query('status == "Эксперт"')
first_purchases_expert = first_purchases_expert[first_purchases_expert["purchase_number"] == 1]
# Считаем, сколько раз встречается каждый тариф
first_distribution_expert = first_purchases_expert["tariff"].value_counts()

# Гость
# Отбираем только первую покупку
first_purchases_quest = ambassadors.copy()
first_purchases_quest = first_purchases_quest.query('status == "Гость"')
first_purchases_quest = first_purchases_quest[first_purchases_quest["purchase_number"] == 1]
# Считаем, сколько раз встречается каждый тариф
first_distribution_quest = first_purchases_quest["tariff"].value_counts()

# Житель
# Отбираем только первую покупку
first_purchases_citizen = ambassadors.copy()
first_purchases_citizen = first_purchases_citizen.query('status == "Житель"')
first_purchases_citizen = first_purchases_citizen[first_purchases_citizen["purchase_number"] == 1]
# Считаем, сколько раз встречается каждый тариф
first_distribution_citizen = first_purchases_citizen["tariff"].value_counts()

print(f"Распределение по первой покупке: Амбассадор")
print(first_distribution_ambassador)
print()
print(f"Распределение по первой покупке: Эксперт")
print(first_distribution_expert)
print()
print(f"Распределение по первой покупке: Гость")
print(first_distribution_quest)
print()
print(f"Распределение по первой покупке: Житель")
print(first_distribution_citizen)

Распределение по первой покупке: Амбассадор
tariff
Покупка тарифа Амбассадор                                  138
ЭКСПЕРТ LIGHT - b2pay - 1 год                               88
Space pass (БВ для партнера)                                18
Билет на Федеральное мероприятие для входа Professional      5
Подписка Житель - 1 месяц                                    3
Name: count, dtype: int64

Распределение по первой покупке: Эксперт
tariff
ЭКСПЕРТ LIGHT - b2pay - 1 год               2368
Покупка тарифа Амбассадор                      1
Подписка Житель - 1 месяц (без списания)       1
Подписка Житель - 1 месяц                      1
Name: count, dtype: int64

Распределение по первой покупке: Гость
tariff
ЭКСПЕРТ LIGHT - b2pay - 1 год                              19
Подписка Житель - 1 месяц (без списания)                    5
Подписка Житель - 1 месяц                                   3
Покупка тарифа Амбассадор                                   3
Space pass (Доп заказ)                    

In [1122]:
# Найдем распределение покупок в зависимости от статуса пользователя
# Амбассадор
# Отбираем только вторую покупку
second_purchases_ambassador = ambassadors.copy()
second_purchases_ambassador = second_purchases_ambassador.query('status == "Амбассадор"')
second_purchases_ambassador = second_purchases_ambassador[second_purchases_ambassador["purchase_number"] == 2]
# Считаем, сколько раз встречается каждый тариф
second_distribution_ambassador = second_purchases_ambassador["tariff"].value_counts()

# Эксперт
# Отбираем только вторую покупку
second_purchases_expert = ambassadors.copy()
second_purchases_expert = second_purchases_expert.query('status == "Эксперт"')
second_purchases_expert = second_purchases_expert[second_purchases_expert["purchase_number"] == 2]
# Считаем, сколько раз встречается каждый тариф
second_distribution_expert = second_purchases_expert["tariff"].value_counts()

# Гость
# Отбираем только вторую покупку
second_purchases_quest = ambassadors.copy()
second_purchases_quest = second_purchases_quest.query('status == "Гость"')
second_purchases_quest = second_purchases_quest[second_purchases_quest["purchase_number"] == 2]
# Считаем, сколько раз встречается каждый тариф
second_distribution_quest = second_purchases_quest["tariff"].value_counts()

# Житель
# Отбираем только вторую покупку
second_purchases_citizen = ambassadors.copy()
second_purchases_citizen = second_purchases_citizen.query('status == "Житель"')
second_purchases_citizen = second_purchases_citizen[second_purchases_citizen["purchase_number"] == 2]
# Считаем, сколько раз встречается каждый тариф
second_distribution_citizen = second_purchases_citizen["tariff"].value_counts()

print(f"Распределение по второй покупке: Амбассадор")
print(second_distribution_ambassador)
print()
print(f"Распределение по второй покупке: Эксперт")
print(second_distribution_expert)
print()
print(f"Распределение по второй покупке: Гость")
print(second_distribution_quest)
print()
print(f"Распределение по второй покупке: Житель")
print(second_distribution_citizen)

Распределение по второй покупке: Амбассадор
tariff
Покупка тарифа Амбассадор                                   99
Space pass (БВ для партнера)                                14
ЭКСПЕРТ LIGHT - b2pay - 1 год                                8
Билет на Федеральное мероприятие для входа Professional      6
Билет STANDARD - Space Business Action 2025 С 20 февраля     5
Билет STANDARD - Space Business Action 2025 С 20 января      4
Скидка на заказ                                              4
Пролонгация Health 2.0                                       1
Билет VIP - Space Business Action 2025 С 20 февраля          1
Билет STANDARD - Space Business Action 2025                  1
Билет VIP - Space Business Action 2025 С 20 января           1
Житель - 1 год                                               1
Name: count, dtype: int64

Распределение по второй покупке: Эксперт
tariff
Space pass (БВ для партнера)     6
ЭКСПЕРТ LIGHT - b2pay - 1 год    4
ЭКСПЕРТ LIGHT - 30 дней          1
Name: count, 

In [1123]:
# Найдем кол-во дней до первой покупки Тарифа Амбассадор

# Для каждого пользователя оставляем только первую покупку (по tariff = 'Покупка тарифа Амбассадор')
first_ambassador_purchase = ambassadors[ambassadors["tariff"] == 
                                        "Покупка тарифа Амбассадор"].groupby("phone")["sale_date"].min()

# Присоединим к датафрейму информацию о первой покупке тарифа Амбассадор
ambassadors_first_ambassador_purchase = ambassadors.merge(first_ambassador_purchase, on="phone", 
                                                          suffixes=("", "_first_ambassador_purchase"))

# Рассчитываем разницу в днях между датой регистрации и первой покупкой тарифа Амбассадор
ambassadors_first_ambassador_purchase["days_to_first_ambassador_purchase"] = \
    (ambassadors_first_ambassador_purchase["sale_date_first_ambassador_purchase"] \
    - ambassadors_first_ambassador_purchase["reg_date"]).dt.days

# Рассчитаем среднее, медианное, максимальное и моду
average_days = ambassadors_first_ambassador_purchase["days_to_first_ambassador_purchase"].mean()
median_days = ambassadors_first_ambassador_purchase["days_to_first_ambassador_purchase"].median()
max_days = ambassadors_first_ambassador_purchase["days_to_first_ambassador_purchase"].max()
mode_days = ambassadors_first_ambassador_purchase["days_to_first_ambassador_purchase"].mode()[0]

print(f"Дней до первой покупки тарифа")
print(f"Среднее количество дней: {round(average_days, 2)}")
print(f"Медианное количество дней: {median_days}")
print(f"Максимальное количество дней: {max_days}")
print(f"Модальное количество дней: {mode_days}")

Дней до первой покупки тарифа
Среднее количество дней: 24.07
Медианное количество дней: 12.0
Максимальное количество дней: 144
Модальное количество дней: 0
